In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
#Read decay tables with Zeroes as NaN
hum = pd.read_csv("hum_MAGs_Abundance.txt", sep='\t', index_col=0, na_values=0.0, header=None)
hum.shape  #15 rows

cow_pig = pd.read_csv("cow_pig_absAbundance.txt", sep='\t', index_col=0, na_values=0.0, header=None)
cow_pig.shape  #28 rows

df = pd.read_csv("Absolute_abundances_all_markers.txt", sep='\t',index_col=0, na_values=0.0, header=None)
df.shape #51 rows

#Break up by those that have 3, 4 or 5 abundance values to estimate decay rate
df3 = df.iloc[:37]  #37 rows
df4 = df.iloc[37:46] #9 rows
df5 = df.iloc[46:]   #5 rows

In [45]:
#Global x variable for all regressions (must be same length as y-variable)
#Format as numpy array
X3 = np.array([0,1,4]).reshape(-1,1)
X4 = np.array([0,1,4,7]).reshape(-1,1)
X5 = np.array([0,1,4,7,14]).reshape(-1,1)
print(X3)

#ln transform every value in table
lnHUM = np.log(hum)
lnCP = np.log(cow_pig)
print(lnHUM.head())

#log base 10 transform every value in the table
logdf3 = np.log10(df3)
logdf4 = np.log10(df4)
logdf5 = np.log10(df5)
logdf3.head()

#Calc Inverse for each value in table. Dont use the 2nd order decay!
#invHUM = 1/hum
#invCP = np.reciprocal(cow_pig)

[[0]
 [1]
 [4]]
                         1          2          3   4
0                                                   
hum1_001 in  H1  16.606676  16.901997  15.006398 NaN
hum1_013 in H1   13.721200  13.835313  13.270783 NaN
hum1_2 in H1     15.243427  15.496338  14.220976 NaN
hum2_001 in H1   15.209277  15.509290  14.513645 NaN
hum3_001 in H1   14.682611  15.048071  14.173185 NaN


,1,2,3,4,5
0,,,,,
HF183 in H1,0.724276,0.758155,0.804139,NaN,NaN
hum1_001 in H1,7.212188,7.340444,6.517196,NaN,NaN
hum1_013 in H1,5.959041,6.008600,5.763428,NaN,NaN
hum1_2 in H1,6.620136,6.729974,6.176091,NaN,NaN
hum2_001 in H1,6.605305,6.735599,6.303196,NaN,NaN


In [79]:
#Make separate arrays for each line in ln() and INVERSE tables where name of array is the index values
y=invCP.iloc[27,:].values

#Do linear regression for each array as y=[A] and X=time. Must be SAME length!
regression = LinearRegression()
regression.fit(X4,y)
y_pred = regression.predict(X4)

#print table with index value from table (i.e. row name), slope, y-intercept,R2, MSE
print(regression.coef_) #slope
print(regression.intercept_) #y-intercept
print(r2_score(y, y_pred))
print(mean_squared_error(y, y_pred))
y_pred.mean()

print(y_pred)

[1.31803017e-05]
-1.0825306299754337e-05
0.8206751908479997
2.846957852903951e-10
[-1.08253063e-05  2.35499538e-06  4.18959004e-05  8.14368055e-05]


1.0000814401215585

In [54]:
#Calculate decay rate and stats for each row in df (No. rows - 1) for 0 indexing
#Using the only D0,D1,and D4 values [i,:3]
regression = LinearRegression()
for i in list(range(0,5)):
    y = logdf5.iloc[i,:5].values
    regression.fit(X5,y)
    y_pred = regression.predict(X5)
    #print(regression.coef_)
    #print(regression.intercept_)
    #print(r2_score(y, y_pred))
    print(mean_squared_error(y, y_pred))

0.05258782654908961
0.04795294200820965
0.08042571373088489
0.4664931994946472
0.14694395942194627


In [138]:
import statsmodels.api as sm

In [143]:
Y2=sm.add_constant(y)
est=sm.OLS(X4,Y2)
est2=est.fit()
est2.summary()

/Users/brittanysuttner/anaconda3/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     6.605
Date:                Fri, 13 Nov 2020   Prob (F-statistic):              0.124
Time:                        16:53:52   Log-Likelihood:                -6.7872
No. Observations:                   4   AIC:                             17.57
Df Residuals:                       2   BIC:                             16.35
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4855      1.104      1.346      0.311      -3.265       6.236
x1          1.427e+04   5550.878      2.570      0.124   -9617.558    3.81e+04
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.857
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.439
Skew:                           0.667   Prob(JB):                        0.803
Kurtosis:                       2.074   Cond. No.                     5.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""